# Введение в нейронные сети

## Урок 5. Рекуррентные нейронные сети

1. Попробуйте починить сеть по словам.

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

Сдавать как обычно гитхабом или гугл колабом.



In [2]:
!pip install -U pip keras tensorflow

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 445.2 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.1 MB 939.4 kB/s eta 0:00:02
   ---------- ----------------------------- 0.3/1.1 MB 1.8 MB/s eta 0:00:01
   ------------------- -------------------- 0.5/1.1 MB 2.6 MB/s eta 0:00:01
   -------------------------------- ------- 0.9/1.1 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 3.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1


In [5]:
!pip install keras_preprocessing

   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   --------- ------------------------------ 10.2/42.6 kB ? eta -:--:--
   -------------------------------------- - 41.0/42.6 kB 393.8 kB/s eta 0:00:01
   ---------------------------------------- 42.6/42.6 kB 414.6 kB/s eta 0:00:00


In [1]:
import re

import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Input, SimpleRNN, Dense, GRU, LSTM, Bidirectional
from keras.utils import to_categorical

In [2]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [3]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [5]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:

#from keras.preprocessing.text import Tokenizer

In [6]:
maxWordsCount = 1000
token = Tokenizer(num_words=maxWordsCount,
                  filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                  lower=True,
                  split=' ',
                  char_level=False)

In [7]:
token.fit_on_texts([text])

In [8]:
token.index_word

{1: 'вы',
 2: 'в',
 3: 'лучший',
 4: 'ответ',
 5: 'на',
 6: 'проблемы',
 7: 'которые',
 8: 'возникли',
 9: 'понедельникдумайте',
 10: 'позитивно',
 11: 'и',
 12: 'верьте',
 13: 'свою',
 14: 'способность',
 15: 'достигать',
 16: 'отличных',
 17: 'результатовесли',
 18: 'смогли',
 19: 'понедельник',
 20: 'подняться',
 21: 'с',
 22: 'постели',
 23: 'значит',
 24: 'супер',
 25: 'герой'}

In [9]:
token.word_index

{'вы': 1,
 'в': 2,
 'лучший': 3,
 'ответ': 4,
 'на': 5,
 'проблемы': 6,
 'которые': 7,
 'возникли': 8,
 'понедельникдумайте': 9,
 'позитивно': 10,
 'и': 11,
 'верьте': 12,
 'свою': 13,
 'способность': 14,
 'достигать': 15,
 'отличных': 16,
 'результатовесли': 17,
 'смогли': 18,
 'понедельник': 19,
 'подняться': 20,
 'с': 21,
 'постели': 22,
 'значит': 23,
 'супер': 24,
 'герой': 25}

In [10]:
dist = list(token.word_counts.items())
dist

[('вы', 3),
 ('лучший', 1),
 ('ответ', 1),
 ('на', 1),
 ('проблемы', 1),
 ('которые', 1),
 ('возникли', 1),
 ('в', 3),
 ('понедельникдумайте', 1),
 ('позитивно', 1),
 ('и', 1),
 ('верьте', 1),
 ('свою', 1),
 ('способность', 1),
 ('достигать', 1),
 ('отличных', 1),
 ('результатовесли', 1),
 ('смогли', 1),
 ('понедельник', 1),
 ('подняться', 1),
 ('с', 1),
 ('постели', 1),
 ('значит', 1),
 ('супер', 1),
 ('герой', 1)]

In [11]:
data = token.texts_to_sequences([text])

In [12]:
data_cat = to_categorical(data[0], num_classes=maxWordsCount)
data_cat.shape

(29, 1000)

In [13]:
inp_words = 3
n = data_cat.shape[0]-inp_words

In [14]:
X = np.array([data_cat[i:i+inp_words, :] for i in range(n)])
Y = data_cat[inp_words:]

In [15]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))

#model.add(GRU(128, activation='tanh'))
#model.add(LSTM(128, activation='tanh'))
#model.add(Bidirectional(SimpleRNN(128, activation='tanh')))

model.add(Dense(maxWordsCount, activation='softmax'))

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 128)            │       144,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1000)           │       129,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 273,512 (1.04 MB)

 Trainable params: 273,512 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

In [18]:
history = model.fit(X, Y, batch_size=32, epochs=20)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 900ms/step - accuracy: 0.0000e+00 - loss: 6.9101
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0000e+00 - loss: 6.8832
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.1154 - loss: 6.8562
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1538 - loss: 6.8288
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3462 - loss: 6.8009
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6923 - loss: 6.7720
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9231 - loss: 6.7419
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9615 - loss: 6.7103
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 6.6768
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 1.0000 - loss: 6.6410
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 6.6027
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss:

In [19]:
def buildPhrase(text, str_len = 20):
    out_str = text
    in_data = token.texts_to_sequences([text])[0]
    for i in range(str_len):
        x = to_categorical(in_data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
        inp = x.reshape(1, inp_words, maxWordsCount)
        pred = model.predict(inp) # предсказываем OHE четвертого символа
        indx = pred.argmax(axis=1)[0]
        in_data.append(indx)
        out_str += " " + token.index_word[indx] # дописываем строку
    return out_str

In [20]:
buildPhrase("вы в понедельник")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'вы в понедельник подняться с постели значит вы супер герой на в которые возникли в понедельникдумайте позитивно и верьте в свою способность достигать'

+